# Lecture 13: Monte Carlo simulations in various ensembles

## Physics 7810, Spring 2020

## 13.1 - Overview

Previously, we introduced the Metropolis Monte Carlo (MC) method, which samples the canonical ($NVT$) ensemble. While this is generally the simplest thermodynamic ensemble to implement in MC simulations, it is not always the most convenient for practical calculations, e.g., of phase equilibria or absorption isotherms. 

Here we discuss methods for carrying out MC simulations in the $NPT$ (isothermal-isobaric) and $\mu VT$ (grand canonical) ensembles, which are two of the most widely used thermodynamic ensembles in computational statistical physics.

## 13.2 - Isothermal-isobaric Monte Carlo

In the constant-$NPT$ ensemble, the configurational average of any property $A({\bf r}^N)$ that depends only on particle coordinates ${\bf r}^N$ is given by

$$
\langle A \rangle_{NPT} = \frac{\int_0^\infty dV \exp(-\beta PV)\ V^N \int d{\bf s}^N A({\bf s}^N; V) \exp \left[ -\beta U({\bf s}^N; V) \right]}{Z^\mathrm{int}(N,P,T)},
$$

where $Z^\mathrm{int}(N,P,T)$ is the configurational part of the isothermal-isobaric partition function introduced previously, and where we make use of *scaled coordinates* ${\bf s}_i = L^{-1} {\bf r}_i$ for $i=1,N$.

Here, the simulation box is assumed to be cubic, with sidelength $L$ and volume $V = L^3$ (note that this treatment is easily generalized to non-cubic boxes). With this scaling, the configurational integral is over the $3N$-dimensional unit cube, and the factor $V^N$ arises from the volume element $d{\bf r}^N$. Writing $A$ and $U$ in terms of scaled coordinates implies an implicit dependence on $V$ (or $L$), e.g., $U({\bf s}^N; V)$.

A Metropolis scheme is implemented in the $NPT$ ensemble by generating a Markov chain of states with a limiting distribution proportional to

$$
\exp\left[ - \beta \left( PV + U({\bf s}^N; V) \right) + N \ln V \right],
$$

using a procedure analogous to that employed for $NVT$ MC simulations. In the constant-$NPT$ MC method, $V$ is simply treated as an additional coordinate, with trial moves

$$
V^\prime = V + \Delta V_\mathrm{max} \left[ 2\ \mathrm{unif}(0,1) - 1 \right]
$$

where $\mathrm{unif}(0,1)$ is a continuous random variable distributed uniformly on the interval $(0,1)$. The maximum volume change $\Delta V_\mathrm{max}$ is a free parameter that can be adjusted to maximize the sampling efficiency of the algorithm, in the same way that the maximum size of single-particle displacement moves is optimized.

The acceptance probability for volume changes is then

$$
P_\mathrm{acc}(o \to n)
= \min \left[1, e^{ - \beta \left[ U({\bf s}^N; V^\prime) - U({\bf s}^N; V) + P (V^\prime - V) - N \beta^{-1} \ln (V^\prime / V) \right] } \right].
$$

This procedure can be shown to sample the correct limiting distribution for the $NPT$ ensemble (see Frenkel & Smit, Section 5.4). The full procedure involves a mixture of single particle displacement trial moves and volume changing moves.

A volume-changing trial move generally requires that we recompute all interparticle interactions (except for simple cases such as power-law pair interactions), and so are significantly more expensive than single-particle displacement  moves. For this reason volume-changing moves are attempted much less frequently than particle displacement moves, say one volume-changing move for every cycle/sweep of $N$ positional moves. To preserve detailed balance, volume-changing moves should not be attempted at regular intervals, but should instead be attempted with a fixed probability (say $1/N$) at every MC step.

## 13.3 - Grand canonical Monte Carlo

Recall that the classical probability density in the $\mu VT$ (*grand canonical*) ensemble is

$$
\rho_{\mu VT}(N,{\bf p}^N,{\bf r}^N) = \frac{1}{Z(\mu,V,T)} \left( \frac{1}{N!} \frac{1}{h^{3N}} e^{\beta \mu N} \exp \left[- \beta H({\bf p}^N,{\bf r}^N) \right] \right),
$$

where $Z(\mu,V,T)$ is the grand partition function,

$$
Z(\mu,V,T) = \sum_{N=0}^\infty \frac{1}{N!} \frac{1}{h^{3N}} e^{\beta \mu N} \int d{\bf p}^N d{\bf r}^N \exp \left[- \beta H({\bf p}^N,{\bf r}^N) \right].
$$

If $H({\bf p}^N,{\bf r}^N) = K({\bf p}^N) + U({\bf r}^N)$, we can integrate out the particle momenta in the usual way to obtain the *configurational* probability density

$$
\rho_{\mu VT}^\mathrm{int}(N,{\bf r}^N) = \frac{1}{Z(\mu,V,T)} \left( \frac{1}{N!} \frac{1}{\Lambda^{3N}} e^{\beta \mu N} \exp \left[- \beta U({\bf r}^N) \right] \right),
$$

where $\Lambda = [h^2 / (2 \pi m k_B T) ]^{1/2}$ is the thermal deBroglie wavelength, and the grand partition function can be written

$$
Z(\mu,V,T) = \sum_{N=0}^\infty \frac{1}{N!} \frac{1}{\Lambda^{3N}} e^{\beta \mu N} \int d{\bf r}^N \exp \left[- \beta U({\bf r}^N) \right].
$$

The configurational probability density can be expressed in terms of scaled coordinates as

$$
\rho_{\mu VT}^\mathrm{int}(N,{\bf s}^N) = \frac{1}{Z(\mu,V,T)} \left( \frac{1}{N!} \frac{1}{\Lambda^{3N}} e^{\beta \mu N} V^N \exp \left[- \beta U({\bf s}^N;V) \right] \right),
$$

with

$$
Z(\mu,V,T) = \sum_{N=0}^\infty \frac{1}{N!} \frac{1}{\Lambda^{3N}} e^{\beta \mu N} V^N \int d{\bf s}^N \exp \left[- \beta U({\bf s}^N;V) \right],
$$

where, as before, the notation $U({\bf s}^N;V)$ indicates and implicit dependence on volume. As for the other ensembles discussed previously, only the configurational (interaction) contributions to thermodynamic properties are calculated during the simulation, and the ideal gas contributions are added afterwards. Because the particle number $N$ fluctuates during the simulation, these contributions will depend on the average number of particles, $\langle N \rangle_{\mu VT}$, which must be calculated during the simulation.

In grand canonical Monte Carlo, we need to construct a Markov chain that correctly samples the this configurational probability density, which is proportional to

$$
\exp \left[ - \beta \left( U({\bf s}^N;V) - \mu N \right) - \ln N! - 3 N \ln \Lambda + N \ln V \right].
$$

There are various ways of doing this, but the standard approach is the one described in Allen & Tildesley and Frenkel & Smit, which involves three types of trial moves:

1. A randomly selected particle is displaced
2. A particle is inserted at a random position in the system
3. A randomly selected particle is removed

As usual, the sequence in which the various types of trial move are attempted should be random, to enforce detailed balance. The average number of attempted particle insertions and removals per MC sweep can be varied to optimize statistical sampling.

The acceptance probability for particle *displacement* moves is the same as for the usual $NVT$ Metropolis method,

$$
P_\mathrm{acc}({\bf s}^N \rightarrow {{\bf s}^\prime}^N) = \min \left[1, \exp \left( - \beta [ U({{\bf s}^\prime}^N;V) - U({\bf s}^N;V) ] \right) \right].
$$

The *insertion* of a new particle is accepted with probability

$$
P_\mathrm{acc}(N \rightarrow N+1) = \min \left[1, \frac{V}{\Lambda^3 (N + 1)} \exp \left(\beta [\mu - U({\bf s}^{N+1};V) + U({\bf s}^N;V) ] \right) \right].
$$

Finally, the *removal* of a particle is accepted with probability

$$
P_\mathrm{acc}(N \rightarrow N-1) = \min \left[1, \frac{\Lambda^3 N }{V} \exp \left(- \beta [\mu + U({\bf s}^{N-1};V) - U({\bf s}^N;V) ] \right) \right].
$$

These acceptance probabilities can be shown to generate the correct limiting distribution (see, e.g., Appendix G in Frenkel and Smit).